# 06 — Task 5: Backtesting vs 60/40 (Aug 1, 2024 → Jul 31, 2025)

In [ ]:
# === Colab Setup ===
!pip install -U pip --quiet
!pip install pandas numpy matplotlib seaborn scikit-learn statsmodels yfinance PyPortfolioOpt arch --quiet
import sys, os, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CWD:", os.getcwd())

In [ ]:
# === Configuration ===
START_DATE = "2015-07-01"
END_DATE   = "2025-07-31"
TICKERS    = ["TSLA", "BND", "SPY"]
BT_START   = "2024-08-01"
BT_END     = "2025-07-31"
TRADING_DAYS = 252
RISK_FREE_DAILY = 0.0

DATA_DIR = "data"
OUT_PLOTS = "outputs/plots"
OUT_MODELS = "outputs/models"
OUT_METRICS = "outputs/metrics"

for d in [DATA_DIR, OUT_PLOTS, OUT_MODELS, OUT_METRICS]:
    os.makedirs(d, exist_ok=True)


In [ ]:
# === Backtest ===
import pandas as pd, numpy as np, matplotlib.pyplot as plt, json

returns = pd.read_csv(f"{DATA_DIR}/returns_daily.csv", index_col=0, parse_dates=True)
with open(f"{OUT_METRICS}/weights_max_sharpe.json") as f:
    w_ms = json.load(f)

bt_rets = returns.loc[BT_START:BT_END, ['TSLA','BND','SPY']].copy()

def port_ret(df, weights):
    w = pd.Series(weights).reindex(df.columns).fillna(0.0)
    return df.mul(w, axis=1).sum(axis=1)

strategy = port_ret(bt_rets, w_ms)
benchmark = 0.6*bt_rets['SPY'] + 0.4*bt_rets['BND']

def cumret(r): return (1 + r).cumprod() - 1
def sharpe_ann(r, rf=0.0, td=252):
    ex = r - rf; mu = ex.mean()*td; sig = ex.std(ddof=0)*np.sqrt(td); 
    return mu/sig if sig>0 else np.nan

cum_s = cumret(strategy)
cum_b = cumret(benchmark)

plt.figure(figsize=(12,5))
plt.plot(cum_s.index, cum_s, label="Strategy (Max Sharpe)")
plt.plot(cum_b.index, cum_b, label="Benchmark 60/40")
plt.title("Cumulative Returns — Strategy vs Benchmark")
plt.legend(); plt.savefig(f"{OUT_PLOTS}/backtest_strategy_vs_benchmark.png"); plt.show()

print(f"Strategy Sharpe: {sharpe_ann(strategy):.3f} | Total Return: {cum_s.iloc[-1]:.2%}")
print(f"Benchmark Sharpe: {sharpe_ann(benchmark):.3f} | Total Return: {cum_b.iloc[-1]:.2%}")
